In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import torch
import torch.nn as nn
import torch.optim as opt
torch.set_printoptions(linewidth=120)
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
import torch.nn.functional as F
from focal import FocalLoss
from sklearn.metrics import roc_auc_score
from efficientnet_pytorch import EfficientNet
import torchvision.transforms as T
from torch.optim.lr_scheduler import ExponentialLR

print(torch.__version__)
torch.manual_seed(0)

1.8.1+cu111


In [2]:
study_label = pd.read_csv('../archive/train_study_level.csv')
paths = []
ids = study_label.id
labels = study_label.iloc[:, 1:].to_numpy()
for i in range(len(ids)):
    paths.append('../archive/study/{x}.png'.format(x = ids[i]))
paths = np.array(paths)

In [3]:
labels = np.argmax(labels, axis=1)
paths.shape, labels.shape

((6054,), (6054,))

In [26]:
'''
# for wb data
training = os.listdir('../train/')
study_label = pd.read_csv('../archive/train_study_level.csv')
image_label = pd.read_csv('../archive/train_image_level.csv')
paths = []
labels = []
dct = pd.read_csv('train.csv', index_col=0)
dct['image'] = dct.image.apply(lambda x: x[:-4])
dct = dct.set_index('study').to_dict()['image']
for index, row in study_label.iterrows():
    name = dct[row['id'].replace('_study', '')] + '.png'
    if name in training:
        paths.append('../train/' + name)
        if row['Negative for Pneumonia'] == 1:
            labels.append(0)
        elif row['Typical Appearance'] == 1:
            labels.append(1)
        elif row['Indeterminate Appearance'] == 1:
            labels.append(2)
        elif row['Atypical Appearance'] == 1:
            labels.append(3)
    else:
        print(name)
'''


"\n# for wb data\ntraining = os.listdir('../train/')\nstudy_label = pd.read_csv('../archive/train_study_level.csv')\nimage_label = pd.read_csv('../archive/train_image_level.csv')\npaths = []\nlabels = []\ndct = pd.read_csv('train.csv', index_col=0)\ndct['image'] = dct.image.apply(lambda x: x[:-4])\ndct = dct.set_index('study').to_dict()['image']\nfor index, row in study_label.iterrows():\n    name = dct[row['id'].replace('_study', '')] + '.png'\n    if name in training:\n        paths.append('../train/' + name)\n        if row['Negative for Pneumonia'] == 1:\n            labels.append(0)\n        elif row['Typical Appearance'] == 1:\n            labels.append(1)\n        elif row['Indeterminate Appearance'] == 1:\n            labels.append(2)\n        elif row['Atypical Appearance'] == 1:\n            labels.append(3)\n    else:\n        print(name)\n"

In [5]:
pd.Series(labels).value_counts(), len(labels)

(1    2855
 0    1676
 2    1049
 3     474
 dtype: int64,
 6054)

In [27]:
np.random.seed(seed=2)
train_idx = np.random.choice(np.arange(6054), size=5000, replace=False)
train_path = np.array(paths)[train_idx]
test_path = np.array(paths)[[x for x in np.arange(6054) if x not in train_idx]]

train_label = np.array(labels)[train_idx]
test_label = np.array(labels)[[x for x in np.arange(6054) if x not in train_idx]]

In [28]:
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4)
model = nn.DataParallel(model, device_ids = [3, 4, 5])

Loaded pretrained weights for efficientnet-b7


In [29]:
trans = T.RandomApply(torch.nn.ModuleList([
                T.RandomAffine(
                    degrees = (10, 30),
                    translate = (0.2, 0.2),
                ),
                T.RandomRotation(degrees=(0, 50)),
                T.RandomHorizontalFlip(p=0.5)  
            ]),p = 0.8)

In [30]:
class classification(nn.Module):
    def __init__(self, paths, labels, size=(512,512), train=False, aug=True):
        self.paths = paths
        self.labels = labels
        self.example = []
        self.size =size
        self.train = train
        self.aug = aug
        
    def __getitem__(self, idx):
        path = self.paths[idx]
        img = cv2.imread(path)
        R, G, B = cv2.split(img)
        output1_R = cv2.equalizeHist(R)
        output1_G = cv2.equalizeHist(G)
        output1_B = cv2.equalizeHist(B)

        img = cv2.merge((output1_R, output1_G, output1_B))
        #img = (img - np.mean(img))/np.std(img)
        img = img/np.mean(img)
        img = cv2.resize(img, self.size)

        x = torch.from_numpy(np.array(img)).view((3, self.size[0], self.size[1]))
        x = x.float()
        y = self.labels[idx]
        y = torch.tensor(y)
        if self.train:
            if self.aug:
                x = trans(x)
            return x, y
        else:
            return x, y
        
    def __len__(self):
        return len(self.paths)

In [31]:
def roc_auc_compute_fn(y_preds: torch.Tensor, y_targets: torch.Tensor) -> float:

    y_true = y_targets.cpu().numpy()
    y_pred = y_preds.cpu().numpy()
    return roc_auc_score(y_true, y_pred, average='weighted', multi_class='ovo')

def one_hot(labels: torch.Tensor, num_classes: int, dtype: torch.dtype = torch.float, dim: int = 1) -> torch.Tensor:
    # if `dim` is bigger, add singleton dim at the end
    if labels.ndim < dim + 1:
        shape = list(labels.shape) + [1] * (dim + 1 - len(labels.shape))
        labels = torch.reshape(labels, shape)

    sh = list(labels.shape)

    if sh[dim] != 1:
        raise AssertionError("labels should have a channel with length equal to one.")

    sh[dim] = num_classes

    o = torch.zeros(size=sh, dtype=dtype, device=labels.device)
    labels = o.scatter_(dim=dim, index=labels.long(), value=1)

    return labels

In [32]:
class Trainer():
    def __init__(self,model,train_set,test_set,opts):
        self.model = model  # neural net
        # device agnostic code snippet
        self.device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
        print(self.device)
        self.model.to(self.device)
    
        self.epochs = opts['epochs']
        if opts['opt'] == 'adam':
            self.optimizer = torch.optim.Adam(model.parameters(), opts['lr'], weight_decay=1e-5, amsgrad=True)
        else:
            self.optimizer = torch.optim.SGD(model.parameters(), opts['lr'], momentum=0.9)
        if opts['loss'] == 'focal':
            self.criterion = FocalLoss(**{"alpha": 0.5, "gamma": 2.0, "reduction": 'mean'})
            self.mix = False
        elif opts['loss'] == 'ce':
            self.criterion = torch.nn.CrossEntropyLoss() 
            self.mix = False
        else:
            self.criterion1 = torch.nn.CrossEntropyLoss() 
            self.criterion2 = FocalLoss(**{"alpha": 0.5, "gamma": 2.0, "reduction": 'mean'})
            self.mix = True
            
        self.train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                                        batch_size=opts['batch_size'],
                                                        shuffle=True)
        self.test_loader = torch.utils.data.DataLoader(dataset=test_set,
                                                       batch_size=opts['batch_size'],
                                                       shuffle=False)
        #self.tb = SummaryWriter(log_dir='./resruns')
        self.best_loss = 1e10
        self.tr_loss = []
        self.sche = ExponentialLR(self.optimizer, gamma=0.1)
        
    def train(self):
        for epoch in range(self.epochs):
            self.model.train() #put model in training mode
            self.tr_loss = []
            for i, (data,labels) in tqdm(enumerate(self.train_loader),
                                                   total = len(self.train_loader)):
                print(labels)
                data, labels = data.to(self.device),labels.to(self.device)
                self.optimizer.zero_grad()  
                outputs = self.model(data)
                if self.mix == False:
                    loss = self.criterion(outputs, labels) 
                    loss.backward()                        
                    self.optimizer.step()                  
                    self.tr_loss.append(loss.item())   
                else:
                    loss = self.criterion1(outputs, labels) + self.criterion2(outputs, labels)
                    loss.backward()                        
                    self.optimizer.step()                  
                    self.tr_loss.append(loss.item()) 
            #self.tb.add_scalar("Train Loss", np.mean(self.tr_loss), epoch)
            self.test(epoch) # run through the validation set
            self.sche.step()
        self.tb.close()
            
    def test(self,epoch):    
        self.model.eval()    # puts model in eval mode - not necessary for this demo but good to know
        self.test_loss = []
        self.test_accuracy = []
        self.predicted = []
        self.true = []
        for i, (data, labels) in enumerate(self.test_loader):

            data, labels = data.to(self.device),labels.to(self.device)

            with torch.no_grad():
                outputs = self.model(data)
            if self.mix == False:
                loss = self.criterion(outputs, labels)
            else:
                loss = self.criterion1(outputs, labels) + self.criterion2(outputs, labels)
            outputs = torch.nn.functional.softmax(outputs, 1)
            _, predicted = torch.max(outputs.data, 1)
            self.test_loss.append(loss.item())
            print(predicted, labels)
            self.test_accuracy.append((predicted == labels).sum().item() / predicted.size(0))
            self.predicted.append(predicted)
            self.true.append(labels)
        
        test_auc = roc_auc_compute_fn(one_hot(torch.cat(self.predicted, dim=0), 4), one_hot(torch.cat(self.true, dim=0), 4))

        print('epoch: {}, train loss: {}, test loss: {}, test accuracy: {}, test auc: {}'.format( 
              epoch+1, np.mean(self.tr_loss), np.mean(self.test_loss), np.mean(self.test_accuracy), test_auc))
        #self.tb.add_scalar("Val Acc", np.mean(self.test_accuracy), epoch)
        #self.tb.add_scalar("Val Loss", np.mean(self.test_loss), epoch)
        if np.mean(self.test_loss) < self.best_loss:
            self.best_loss = np.mean(self.test_loss)
            #torch.save(self.model.state_dict(), './model_weights/resbest.pt')

In [33]:
train_set = classification(train_path[:200], train_label[:200], size=(512, 512), train=True, aug=False)
val_set = classification(test_path[:10], train_label[:10], size=(512, 512), train=False, aug=False)

In [34]:
pd.Series(train_label[:50]).value_counts()

1    25
0    16
2     7
3     2
dtype: int64

In [35]:
opts = {
    'lr': 1e-5,
    'epochs': 50,
    'batch_size': 20,
    'opt': 'adam',
    'loss': 'ce'
}
train = Trainer(model, train_set, val_set, opts)
train.train()

cuda:3


tensor([1, 1, 0, 2, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 2, 1, 0, 2, 1, 0])
tensor([1, 0, 0, 1, 2, 1, 0, 1, 1, 1, 2, 0, 2, 1, 0, 1, 1, 1, 1, 1])
tensor([1, 2, 1, 3, 2, 0, 0, 0, 1, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0])
tensor([0, 1, 0, 1, 3, 1, 0, 1, 1, 0, 1, 1, 0, 0, 2, 0, 0, 1, 1, 1])
tensor([1, 1, 1, 1, 0, 3, 2, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 3, 1, 1])
tensor([2, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 2, 0, 2, 1, 1, 1, 1, 0, 1])
tensor([1, 1, 3, 1, 0, 1, 2, 3, 2, 0, 0, 1, 0, 2, 0, 3, 1, 2, 2, 1])
tensor([0, 1, 1, 2, 0, 0, 1, 1, 1, 1, 0, 1, 0, 3, 1, 2, 1, 2, 3, 1])
tensor([1, 1, 2, 0, 1, 1, 1, 0, 0, 2, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 3, 0, 0, 1, 1, 1, 2, 1, 1, 1])

tensor([2, 2, 2, 2, 2, 1, 2, 3, 3, 3], device='cuda:3') tensor([2, 3, 1, 1, 0, 1, 0, 0, 0, 1], device='cuda:3')
epoch: 1, train loss: 1.3686195731163024, test loss: 1.398083209991455, test accuracy: 0.2, test auc: 0.5555555555555556


tensor([0, 1, 1, 1, 0, 0, 1, 2, 0, 2, 0, 0, 0, 2, 1, 2, 2, 1, 1, 0])
tensor([1, 1, 2, 1, 0, 3, 1, 0, 0, 1, 1, 1, 0, 3, 2, 1, 0, 1, 1, 1])
tensor([1, 2, 0, 1, 0, 2, 0, 0, 1, 1, 2, 1, 1, 2, 1, 0, 0, 1, 1, 2])
tensor([1, 0, 0, 1, 1, 1, 1, 2, 2, 0, 2, 0, 0, 1, 1, 1, 3, 1, 1, 1])
tensor([0, 2, 1, 2, 0, 1, 0, 1, 1, 0, 2, 3, 3, 1, 1, 1, 2, 1, 0, 1])
tensor([2, 1, 1, 0, 1, 3, 0, 0, 0, 1, 3, 1, 1, 0, 1, 0, 1, 0, 1, 1])
tensor([1, 0, 2, 1, 0, 1, 1, 1, 0, 3, 2, 2, 0, 1, 1, 1, 0, 3, 1, 1])
tensor([0, 1, 1, 1, 2, 0, 1, 0, 1, 1, 2, 1, 0, 1, 1, 0, 1, 3, 2, 0])
tensor([1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 1, 1, 0, 1, 1])
tensor([0, 1, 0, 1, 1, 2, 2, 1, 1, 0, 2, 0, 1, 1, 0, 0, 1, 1, 1, 1])

tensor([2, 2, 2, 2, 2, 1, 2, 2, 2, 3], device='cuda:3') tensor([2, 3, 1, 1, 0, 1, 0, 0, 0, 1], device='cuda:3')
epoch: 2, train loss: 1.3603318214416504, test loss: 1.4008502960205078, test accuracy: 0.2, test auc: 0.5555555555555556


tensor([1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 2, 2, 1, 2, 0, 2])
tensor([1, 0, 1, 0, 1, 1, 2, 1, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 2, 0, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 1])
tensor([2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 0, 2, 1, 1, 1, 0])
tensor([2, 1, 1, 0, 0, 1, 2, 1, 1, 0, 0, 2, 0, 2, 0, 3, 1, 1, 1, 1])
tensor([3, 3, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 0, 3])
tensor([2, 2, 1, 1, 3, 0, 1, 0, 3, 0, 1, 1, 2, 1, 1, 2, 1, 2, 0, 0])
tensor([1, 0, 0, 1, 2, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 3, 0, 1, 2, 1])
tensor([0, 3, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 3, 0, 1])
tensor([0, 0, 0, 0, 1, 2, 1, 3, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1])

tensor([2, 2, 3, 2, 2, 1, 2, 3, 2, 3], device='cuda:3') tensor([2, 3, 1, 1, 0, 1, 0, 0, 0, 1], device='cuda:3')
epoch: 3, train loss: 1.347407627105713, test loss: 1.4005681276321411, test accuracy: 0.2, test auc: 0.5555555555555556


tensor([0, 2, 2, 2, 1, 0, 2, 1, 2, 2, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1])
tensor([1, 1, 0, 2, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 0, 0, 1, 2])
tensor([2, 2, 0, 2, 0, 1, 0, 0, 1, 1, 2, 2, 1, 1, 0, 1, 1, 1, 1, 1])
tensor([0, 1, 1, 0, 1, 0, 2, 1, 0, 1, 0, 1, 2, 2, 1, 1, 1, 0, 0, 0])
tensor([1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 0, 3, 3, 1, 2, 1, 0, 1, 2])
tensor([1, 1, 2, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1])
tensor([0, 0, 1, 1, 1, 0, 1, 1, 1, 3, 1, 1, 0, 0, 1, 3, 0, 0, 1, 1])
tensor([1, 3, 0, 1, 3, 2, 0, 1, 3, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 2])



KeyboardInterrupt: 

In [ ]:
print(model)